In [54]:
import sys
import os
import subprocess
from collections import namedtuple
from typing import List, Union, Optional, Tuple, Dict, Any

# from _get_cincyconda_doc import get_docstring

CONDA_INSTALL_PATH = "/rsystem/Rapps/anaconda310/bin/conda"

class CondaEnv:
    def __init__(self,
                 name: str = ".env",
                 path: str = None,
                 packages: list = None,
                 conda_install_path: str = CONDA_INSTALL_PATH,
                 shell: str = 'bash',
                 channel_alias: str = None,
                 ):
        self.name = name
        self.path = path
        self.packages = packages
        self.conda = conda_install_path
        self.shell = shell
        self.channel_alias = channel_alias

        self.conda_envs = None

    def __post_init__(self):
        # if the base env is not activated, activate it
        self._activate_base()

        # if jupyter, notebook, ipykernel are not passed in, add them
        if 'jupyter' not in self.packages:
            self.packages.append('jupyter')
        if 'notebook' not in self.packages:
            self.packages.append('notebook')
        if 'ipykernel' not in self.packages:
            self.packages.append('ipykernel')

        

    def __str__(self):
        return f"CondaEnv({self.name})"

    def __repr__(self):
        return f"CondaEnv({self.name})"

    def _activate_base(self):
        # if the base env is not activated, activate it
        # there is no CONDA_DEFAULT_ENV available
        # so we have to check the path
        assert hasattr(self, 'conda'), \
            f"conda_install_path is not set. Please set it to the path of your conda \
                installation. Try '{CONDA_INSTALL_PATH}'"
        assert hasattr(sys, 'base_prefix'), \
            f"sys.base_prefix: '{sys.base_prefix}' is not set"

        os.system(f"{self.conda} activate base")

    def Setup(self):
        """
        Does the unique setup steps for setting up anaconda on the jupyterhub server.
        """
        # initialize conda in the user's shell
        self.init(help=False)

        # close and re-open the shell
        subprocess.run([f'{self.shell}', '-c', 'exit'])

        # list the commands to run
        cmd = [
            # clear the conda cache and channel list
            f"{self.conda} clean --all",

            # add the CIC repo
            f"{self.conda} config --set channel_alias {self.channel_alias}/repo",

            # add the restricted_channel to default channels
            f"{self.conda} config --prepend channels default_channels restricted_channel",

            # add the CIC repo to the default channels
            f"{self.conda} repo config --set sites.anaconda.url {self.channel_alias}",

            # set anaconda to be the default site
            f"{self.conda} repo config --set default_site anaconda",

            # log in
            f"{self.conda} repo login"
        ]

        # loop through the commands and run them
        for c in cmd:
            subprocess.run(c.split())

    def Init(self,
             help: bool = False) -> None:
        """
        Initializes conda in the user's shell

        Parameters
        ----------
        shell : str, optional
            The shell to initialize conda in, by default 'bash'
        help : bool, optional
            Whether to print the help message, by default False

        Returns
        -------
        None

        Raises
        ------
        AssertionError
            If the shell is not supported

        Example Usage
        -------------
        >>> from CincyConda import CondaEnv
        >>> env = CondaEnv(name='test', packages=['numpy', 'pandas'])
        >>> env.init(shell='bash')
        
        >>> # expected output:
        >>> #  ==> For changes to take effect, close and re-open your current shell. <==
        >>> #  ==> If you'd prefer that conda's base environment not be activated on startup,
        >>> #      set the auto_activate_base parameter to false: <==
        >>> #  conda config --set auto_activate_base false
        """
        # check the shell input
        assert self.shell in ['bash', 'zsh', 'fish', 'powershell', 'xonsh'], \
            f"shell: '{self.shell}' is not supported. Please use one of the following: \
             ['bash', 'zsh', 'fish', 'powershell', 'xonsh']"

        if help:
            msg = os.system(f"{self.conda} init --help")
        else:
            msg = os.system(f"{self.conda} init {self.shell}")

        print(msg)

    def Create(self,
               help:bool = False) -> None:
        # create the env
        os.system(f"{self.conda} create --prefix {self.path} -y {' '.join(self.packages)}")

        # create the kernel
        os.system(f"python -m ipykernel install --user --name {self.name} --display-name {self.name}")

        # if the base env is not activated, activate it
        self._activate_base()

    def Remove(self, env:CondaEnvs):
        

    def update(self, package):
        # if the base env is not activated, activate it
        self._activate_base()

        # update the env
        os.system(f"{self.conda} update --prefix {self.path} {package} -y")

        # if the base env is not activated, activate it
        self._activate_base()

    def install(self, package):
        # if the base env is not activated, activate it
        self._activate_base()

        # install the package
        os.system(f"{self.conda} install --prefix {self.path} {package} -y")

        # if the base env is not activated, activate it
        self._activate_base()

    def _all_envs(self) -> list:
        """
        Returns the available conda environments in a python list

        Parameters
        ----------
        name : str
            The name of the conda environment to return

        Returns
        -------
        list
            A list of available conda environments

        Raises
        ------
        AssertionError
            If the name is not a string
        AssertionError
            If the name is not in the list of available conda environments, and is
            not a substring of any of the available conda environment names or paths

        """
        # get the envs
        # envs = os.system(f"{self.conda} env list")

        if self.conda_envs is not None:
            return self.conda_envs
        else:
            envs = subprocess.run([f"{self.conda}", "env", "list"],
                                  stdout=subprocess.PIPE,
                                  stderr=subprocess.PIPE)

            # convert to a list
            if envs.returncode != 0:
                print(f"Error executing command: {envs.stderr.decode('utf-8')}")
                return []
            else:
                self.conda_envs = namedtuple('CondaEnvs', ['name', 'path'])   
                output = []
                for line in envs.stdout.decode('utf-8').splitlines()[2:]:
                    env_name = line.split()
                    env_path = line.split()
                    output.append(self.conda_envs(env_name, env_path))
            self.conda_envs = output
            return output

    

    def env(self,
            name: str = None) -> Union[list, str]:
        """
        Returns the available conda environments in a python list

        Parameters
        ----------
        name : str
            The name of the conda environment to return

        Returns
        -------
        list
            A list of available conda environments

        Raises
        ------
        AssertionError
            If the name is not a string
        AssertionError
            If the name is not in the list of available conda environments, and is
            not a substring of any of the available conda environment names or paths

        """
        # get the envs
        envs = self._all_envs()

        # if no name is provided, return all envs
        if name is None:
            return envs

        # if a name is provided, make sure it is a string
        assert isinstance(name, str), \
            f"name must be a string, not {type(name)}"
        
        # if the name is in the list of CondaEnvs objects, return it
        if name in [env.name for env in envs]:
            return [env for env in envs if env.name == name][0]
        
        # if the name is not in the list of CondaEnvs objects, check if it is a substring
        # of any of the names first, then the paths
        elif name not in [env.name for env in envs]:
            envs = [env for env in envs if name in env.name]
            if len(envs) == 1:
                return envs[0]
            elif len(envs) > 1:
                return envs
            else:
                envs = [env for env in envs if name in env.path]
                if len(envs) == 1:
                    return envs[0]
                elif len(envs) > 1:
                    return envs
                else:
                    raise AssertionError(f"Could not find environment: {name}")

In [55]:
c = CondaEnv()
c.env()

[CondaEnvs(name=['/rdata/aweaver'], path=['/rdata/aweaver']),
 CondaEnvs(name=['/rdata/aweaver/hitratio/.env'], path=['/rdata/aweaver/hitratio/.env']),
 CondaEnvs(name=['base', '/rsystem/Rapps/anaconda310'], path=['base', '/rsystem/Rapps/anaconda310']),
 CondaEnvs(name=[], path=[])]

In [58]:
c.env('base').path

['base', '/rsystem/Rapps/anaconda310']

In [5]:
os.getcwd()

'/rdata/aweaver'

In [10]:
print(c.env())

# conda environments:
#
                         /rdata/aweaver
                         /rdata/aweaver/hitratio/.env
base                     /rsystem/Rapps/anaconda310

0


In [12]:
import sys
hasattr(sys, 'real_prefix')

False

In [13]:
hasattr(sys, 'base_prefix')

True

In [16]:
sys.prefix

'/rdata/aweaver/EGSB/aw/aw-env'

In [15]:
sys.base_prefix

'/rdata/aweaver/EGSB/aw/aw-env'

In [19]:
c = CondaEnv()
os.system(f"{c.conda} env list")
c.env()

# conda environments:
#
                         /rdata/aweaver
                         /rdata/aweaver/hitratio/.env
base                     /rsystem/Rapps/anaconda310




CommandNotFoundError: Your shell has not been properly configured to use 'conda activate'.
To initialize your shell, run

    $ conda init <SHELL_NAME>

Currently supported shells are:
  - bash
  - fish
  - tcsh
  - xonsh
  - zsh
  - powershell

See 'conda init --help' for more information and options.

IMPORTANT: You may need to close and restart your shell after running 'conda init'.



CommandNotFoundError: Your shell has not been properly configured to use 'conda activate'.
To initialize your shell, run

    $ conda init <SHELL_NAME>

Currently supported shells are:
  - bash
  - fish
  - tcsh
  - xonsh
  - zsh
  - powershell

See 'conda init --help' for more information and options.

IMPORTANT: You may need to close and restart your shell after running 'conda init'.




'# conda environments:\n#\n                         /rdata/aweaver\n                         /rdata/aweaver/hitratio/.env\nbase                     /rsystem/Rapps/anaconda310\n\n'